## Logistic Regression

In [2]:
%matplotlib inline

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
    
import statsmodels
import statsmodels.api as sm

from nose.tools import assert_equal
from numpy.testing import assert_array_equal, assert_array_almost_equal
from pandas.util.testing import assert_frame_equal

sns.set(style="white", font_scale=2.0)

In [3]:
filename = '2001.csv'

usecols = (3, 8, 15, 17)
columns = ['DayOfWeek', 'UniqueCarrier', 'DepDelay', 'Origin']

all_data = pd.read_csv(filename, header=0, na_values=['NA'], usecols=usecols, names=columns).dropna()

local = all_data.loc[all_data['Origin'] == 'ORD'].dropna()

In [32]:
print(local.head())

      DayOfWeek UniqueCarrier  DepDelay Origin
1855          2            US      -1.0    ORD
1856          3            US      -4.0    ORD
1857          4            US      -3.0    ORD
1858          5            US      -3.0    ORD
1859          6            US      -4.0    ORD


### Convert DepDelay to binary

In [4]:
def convert_to_binary(df, column, cutoff):
    '''
    Converts one column in Pandas.DataFrame "df" into binary
    as a new column, and returns the new DataFrame ("df" plus new binary column).
    Note that "df" should NOT be altered.
    
    The returned DataFrame has one more column than "df".
    The name of this column is in the form "column_binary".
    For example, if "column" is "DepDelay", the name of the extra column
    in the returned DataFrame is "DepDelay_binary".
    
    We assume that df[column] contains only ints or floats.
    If df[column] < cutoff, df[column_binary] is 0.
    If df[column] >= cutoff, df[column_binary] is 1.
    
    Parameters
    ----------
    df: A Pandas.DataFrame.
    column: A string.
    cutoff: An int.
    
    Returns
    -------
    A Pandas.DataFrame.
    '''
    
    new=df.copy()
    new[column+'_binary']=(new[column] >= cutoff).astype(int)
    return new

In [5]:
local = convert_to_binary(local, 'DepDelay', 5)

In [62]:
print(local.tail(10))

         DayOfWeek UniqueCarrier  DepDelay Origin  DepDelay_binary
5960735          6            DL       4.0    ORD                0
5960736          7            DL       7.0    ORD                1
5960737          1            DL      -2.0    ORD                0
5960738          2            DL      -3.0    ORD                0
5960739          3            DL       0.0    ORD                0
5960740          4            DL      58.0    ORD                1
5960741          5            DL       1.0    ORD                0
5960742          6            DL       0.0    ORD                0
5960743          7            DL      -8.0    ORD                0
5960744          1            DL      -3.0    ORD                0


In [6]:
df0 = pd.DataFrame({
    'a': list(range(-5, 5)),
    'b': list(range(10))
    })

test1 = convert_to_binary(df0, 'a', 0)
answer1 = df0.join(pd.DataFrame({'a_binary': [0] * 5 + [1] * 5}))
assert_frame_equal(test1, answer1)

test2 = convert_to_binary(df0, 'b', 4)
answer2 = df0.join(pd.DataFrame({'b_binary': [0] * 4 + [1] * 6}))
assert_frame_equal(test2, answer2)

### Convert categorical variables to dummy indicator variables

In [31]:
def convert_to_dummy(df, dummy_columns, keep_columns):
    '''
    Transforms categorical variables of dummy_columns into binary indicator variables.
    
    Parameters
    ----------
    df: A pandas.DataFrame
    dummy_columns: A list of strings. Columns of df that are converted to dummies.
    keep_columns: A list of strings. Columns of df that are kept in the result.
    
    
    Returns
    -------
    A pandas.DataFrame
    '''
    
    new=df[keep_columns+dummy_columns]
    new=pd.get_dummies(new,columns=dummy_columns)
    return new

In [32]:
data = convert_to_dummy(local, dummy_columns=['DayOfWeek', 'UniqueCarrier'], keep_columns=['DepDelay_binary'])
print(data.head())

      DepDelay_binary  DayOfWeek_1  DayOfWeek_2  DayOfWeek_3  DayOfWeek_4  \
1855                0            0            1            0            0   
1856                0            0            0            1            0   
1857                0            0            0            0            1   
1858                0            0            0            0            0   
1859                0            0            0            0            0   

      DayOfWeek_5  DayOfWeek_6  DayOfWeek_7  UniqueCarrier_AA  \
1855            0            0            0                 0   
1856            0            0            0                 0   
1857            0            0            0                 0   
1858            1            0            0                 0   
1859            0            1            0                 0   

      UniqueCarrier_AS  UniqueCarrier_CO  UniqueCarrier_DL  UniqueCarrier_HP  \
1855                 0                 0                 0        

### Add intercept

In [50]:
def add_intercept(df):
    '''
    Appends to "df" an "Intercept" column whose values are all 1.0.
    Note that "df" should NOT be altered.
    
    Parameters
    ----------
    df: A pandas.DataFrame
    
    Returns
    -------
    A pandas.DataFrame
    '''
    
    new=df.copy()
    new['Intercept']=1
    return new

In [51]:
data = add_intercept(data)
print(data['Intercept'].head())

1855    1
1856    1
1857    1
1858    1
1859    1
Name: Intercept, dtype: int64


In [52]:
df0 = pd.DataFrame({'a': [c for c in 'abcde']})

test1 = add_intercept(df0)
answer1 = df0.join(pd.DataFrame({'Intercept': [1] * 5}))

assert_frame_equal(test1, answer1)

### Function: fit_logistic()

In [62]:
def fit_logitistic(df, train_columns, test_column):
    '''
    Fits a logistic regression model on "train_columns" to predict "test_column".
    
    The function returns a tuple of (model ,result).
    "model" is an instance of Logit(). "result" is the result of Logit.fit() method.
    
    Parameters
    ----------
    train_columns: A list of strings
    test_column: A string
    
    Returns
    -------
    A tuple of (model, result)
    model: An object of type statsmodels.discrete.discrete_model.Logit
    result: An object of type statsmodels.discrete.discrete_model.BinaryResultsWrapper
    '''
    
    logit = sm.Logit(data[test_column], data[train_columns])
    result = logit.fit()
    return logit,result

In [63]:
train_columns = [
        'DayOfWeek_2', 'DayOfWeek_3', 'DayOfWeek_4',
        'DayOfWeek_5', 'DayOfWeek_6', 'DayOfWeek_7',
        'UniqueCarrier_AS', 'UniqueCarrier_CO', 'UniqueCarrier_DL',
        'UniqueCarrier_HP', 'UniqueCarrier_MQ', 'UniqueCarrier_NW',
        'UniqueCarrier_TW', 'UniqueCarrier_UA', 'UniqueCarrier_US',
        'Intercept'
        ]

model, result = fit_logitistic(data, train_columns=train_columns, test_column='DepDelay_binary')

Optimization terminated successfully.
         Current function value: 0.589094
         Iterations 5


In [64]:
print(result.summary())

                           Logit Regression Results                           
Dep. Variable:        DepDelay_binary   No. Observations:               321227
Model:                          Logit   Df Residuals:                   321211
Method:                           MLE   Df Model:                           15
Date:                Wed, 19 Jul 2017   Pseudo R-squ.:                0.005735
Time:                        18:33:49   Log-Likelihood:            -1.8923e+05
converged:                       True   LL-Null:                   -1.9032e+05
                                        LLR p-value:                     0.000
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
DayOfWeek_2         -0.1574      0.015    -10.479      0.000      -0.187      -0.128
DayOfWeek_3          0.0164      0.015      1.113      0.266      -0.012       0.045
DayOfWeek_4          0.2148 

In [65]:
assert_equal(isinstance(model, statsmodels.discrete.discrete_model.Logit), True)
assert_equal(isinstance(result, statsmodels.discrete.discrete_model.BinaryResultsWrapper), True)

assert_equal(model.exog_names, train_columns)
assert_equal(model.endog_names, 'DepDelay_binary')

assert_array_equal(model.exog, data[train_columns].values)
assert_array_equal(model.endog, data['DepDelay_binary'].values)

test_conf_int = result.conf_int()
answer_conf_int = pd.DataFrame(
    index=train_columns,
    data={
        0: np.array([
            -0.18681953, -0.01247828,  0.18652782,  0.17760447, -0.00675086,
            0.07974488, -0.6227236 , -0.06873794,  0.50352299,  0.78551841,
            0.06694527,  0.21153022,  0.30383117,  0.17150234,  0.20497387,
            -1.166157  ]),
        1: np.array([
            -0.12794527,  0.04524193,  0.24298324,  0.23413964,  0.05254801,
            0.13724129, -0.09649653,  0.04848345,  0.59783265,  0.93824414,
            0.11429806,  0.30780938,  0.44591082,  0.20879553,  0.30969833,
            -1.11899193])
        }
    )
assert_frame_equal(test_conf_int, answer_conf_int)

In [66]:
print(local.groupby('DayOfWeek').mean().sort_values(by='DepDelay', ascending=False))

            DepDelay  DepDelay_binary
DayOfWeek                            
4          11.419251         0.311135
5          11.306297         0.309324
7          10.244282         0.288786
3           9.962156         0.270426
1           9.336543         0.267238
6           9.015426         0.271879
2           7.230843         0.237644


In [67]:
print(local.groupby('UniqueCarrier').mean().sort_values(by='DepDelay', ascending=False))

               DayOfWeek   DepDelay  DepDelay_binary
UniqueCarrier                                       
HP              3.973684  18.245494         0.444845
DL              3.972141  11.719453         0.370235
UA              3.953615  11.027225         0.291036
TW              3.881535  10.183537         0.330645
NW              3.827438   9.445194         0.305562
CO              3.788118   9.215042         0.251857
US              3.922626   9.207263         0.305168
MQ              3.937944   9.099671         0.270765
AA              3.957042   8.298827         0.253498
AS              3.991667   4.791667         0.191667
